## Run this first if you want NDVI/NDWI basemaps

In [35]:
import ee
ee.Authenticate(force=True)



Successfully saved authorization token.


## Vibes

In [1]:
%load_ext autoreload
%autoreload 2

from geovibes.ui import GeoVibes

vibes = GeoVibes(
    duckdb_directory = '/Users/christopherren/geovibes/local_databases/',
    start_date = '2024-01-01',
    end_date = '2025-01-01',
    enable_ee = False,
    verbose=False)